In [1]:
import awkward as ak
import dask
import dask_awkward
from distributed import performance_report
import hist
from hist.dask import Hist
import json
from coffea import processor
from coffea.nanoevents import BaseSchema, PFNanoAODSchema, NanoAODSchema
from coffea.dataset_tools import apply_to_dataset, apply_to_fileset, preprocess
from coffea.dataset_tools import max_chunks, max_files, slice_chunks, slice_files
from coffea.analysis_tools import Weights, PackedSelection
import matplotlib.pyplot as plt
from coffea.nanoevents.methods import vector
from coffea.nanoevents import NanoEventsFactory

/tmp/ipykernel_2328979/3704368154.py:14: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [2]:
import fsspec_xrootd
import dask.multiprocessing
import os

In [3]:
from msdprocessor import msdProcessor

In [4]:
from distributed import Client
from lpcjobqueue import LPCCondorCluster


Condor configuration not found! run the following command outside the apptainer instance
grep -v '^include' /etc/condor/config.d/01_cmslpc_interactive > .condor_config
File specified in CONDOR_CONFIG environment variable:
"/srv/.condor_config" does not exist.

Neither the environment variable CONDOR_CONFIG,
/etc/condor/, /usr/local/etc/, nor ~condor/ contain a condor_config source.
Either set CONDOR_CONFIG to point to a valid config source,
or put a "condor_config" file in /etc/condor/ /usr/local/etc/ or ~condor/


In [5]:
client= Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 8,Total memory: 11.43 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39267,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 8
Started: Just now,Total memory: 11.43 GiB
Comm: tcp://127.0.0.1:40343,Total threads: 2
Dashboard: /proxy/39779/status,Memory: 2.86 GiB
Nanny: tcp://127.0.0.1:38957,


In [6]:
# with open("2018_QCD_Pt1000to1400.json", "rt") as file:
#     initial_fileset = json.load(file)

In [7]:
qcd_slice = "Pt3200toInf"

In [8]:
with open("QCDcuts/QCD_"+qcd_slice+".json", "rt") as file:
    initial_fileset = json.load(file)

In [9]:
files={}
for i, j in initial_fileset.items():
    print(f"this is i {i}")
    files[i]={"files":{}}
    for k in j:
        # k = k.replace("cmsxrootd.fnal.gov", "xcache")
        files[i]["files"][k] = "Events"
        print(f"this is k {k}")
    

this is i QCD_Pt_3200toInf
this is k root://cmsxrootd.fnal.gov//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/QCD_Pt_3200toInf/211111_021344/0000/nano_mc2018_1-1.root
this is k root://cmsxrootd.fnal.gov//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/QCD_Pt_3200toInf/211111_021344/0000/nano_mc2018_1-2.root
this is k root://cmsxrootd.fnal.gov//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/QCD_Pt_3200toInf/211111_021344/0000/nano_mc2018_1-3.root
this is k root://cmsxrootd.fnal.gov//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/QCD_Pt_3200toInf/211111_021344/0000/nano_mc2018_1-4.root
this is k root://cmsxrootd.fnal.gov//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/QCD_Pt_3200toInf/211111_021344/0000/nano_mc2018_1-5.root
this is k root://cmsxrootd.fnal.gov//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_3200to

In [10]:
preprocessed_available, preprocessed_total = preprocess(
        files,
        step_size=50_000,
        align_clusters=None,
        skip_bad_files=True,
        recalculate_steps=False,
        files_per_batch=1,
        file_exceptions=(OSError,),
        save_form=True,
        uproot_options={},
        step_size_safety_factor=0.5,
    )

In [11]:
print(preprocessed_available.keys())

dict_keys(['QCD_Pt_3200toInf'])


In [12]:
import gzip, pickle, json
output_file = "scaleout_fileset"
with gzip.open(f"{output_file}_available.json.gz", "wt") as file:
    json.dump(preprocessed_available, file, indent=2)
    print(f"Saved available fileset chunks to {output_file}_available.json.gz")
with gzip.open(f"{output_file}_all.json.gz", "wt") as file:
    json.dump(preprocessed_total, file, indent=2)
    print(f"Saved complete fileset chunks to {output_file}_all.json.gz")

Saved available fileset chunks to scaleout_fileset_available.json.gz
Saved complete fileset chunks to scaleout_fileset_all.json.gz


In [13]:
test_preprocessed_files = max_files(preprocessed_available, 2)
test_preprocessed = max_chunks(test_preprocessed_files, 30)

In [14]:
print(test_preprocessed_files)

{'QCD_Pt_3200toInf': {'files': {'root://cmsxrootd.fnal.gov:1094//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/QCD_Pt_3200toInf/211111_021344/0000/nano_mc2018_1-1.root': {'object_path': 'Events', 'steps': [[0, 20000]], 'num_entries': 20000, 'uuid': 'd9ce945c-44de-11ec-9ad0-020013acbeef'}, 'root://cmsxrootd.fnal.gov:1094//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/QCD_Pt_3200toInf/211111_021344/0000/nano_mc2018_1-2.root': {'object_path': 'Events', 'steps': [[0, 20000]], 'num_entries': 20000, 'uuid': 'bfd65c0a-45ac-11ec-9165-1c0b030abeef'}}, 'form': '...', 'metadata': None}}


In [15]:
compute, results = apply_to_fileset(data_manipulation=msdProcessor(),    
                            fileset=test_preprocessed_files,
                            schemaclass=PFNanoAODSchema,
                            uproot_options={"allow_read_errors_with_report": (OSError, KeyError)},
                           )

In [16]:
compute_results=dask.compute(compute)

/uscms/home/nyabando/nobackup/miniconda3/envs/fastjet/lib/python3.12/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/uscms/home/nyabando/nobackup/miniconda3/envs/fastjet/lib/python3.12/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors 

#--------------------------------------------------------------------------
#                         FastJet release 3.4.1
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


/uscms/home/nyabando/nobackup/miniconda3/envs/fastjet/lib/python3.12/site-packages/awkward/_nplikes/array_module.py:246: RuntimeWarning: invalid value encountered in sqrt
  return impl(*broadcasted_args, **(kwargs or {}))


#--------------------------------------------------------------------------
#                         FastJet release 3.4.1
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------
#----------------------------------------------------------------------

In [17]:
print(compute_results)

({'QCD_Pt_3200toInf': {'ExampleHistogram': Hist(
  Regular(36, 0, 252, name='msoftdrop', label='Jet msoftdrop'),
  Regular(10, 0, 1, name='n2', label='Jet n2'),
  storage=Weight()) # Sum: WeightedSum(value=32587, variance=32587) (WeightedSum(value=40000, variance=40000) with flow), 'ExampleHistogram1': Hist(
  Regular(10, 0, 1, name='n2b', label='Jet n2b'),
  Regular(36, 0, 252, name='msoftdrop1', label='Jet msoftdrop'),
  storage=Weight()) # Sum: WeightedSum(value=32170, variance=32170) (WeightedSum(value=40000, variance=40000) with flow), 'ExampleHistogram2': Hist(
  Regular(10, 0, 1, name='n2b2', label='Jet n2b2'),
  Regular(36, 0, 252, name='msoftdrop2', label='Jet msoftdrop'),
  storage=Weight()) # Sum: WeightedSum(value=30810, variance=30810) (WeightedSum(value=40000, variance=40000) with flow), 'ExampleHistogram3': Hist(
  Regular(10, 0, 1, name='n2b3', label='Jet n2b3'),
  Regular(36, 0, 252, name='msoftdrop3', label='Jet msoftdrop'),
  storage=Weight()) # Sum: WeightedSum(valu

In [18]:
from coffea import processor, util

In [19]:
util.save(compute_results, "coffea/QCD_"+qcd_slice+".coffea")

In [20]:
# util.save(compute_results, "2018_QCD_Pt1000to1400.coffea")

In [20]:
# print(compute_results)

In [22]:
h=compute_results[0]["QCD_Pt_1000to1400"]["ExampleHistogram"]

In [23]:
print(h)

Hist(
  Regular(36, 0, 252, name='msoftdrop', label='Jet msoftdrop'),
  Regular(10, 0, 1, name='n2', label='Jet n2'),
  storage=Weight()) # Sum: WeightedSum(value=19934, variance=19934) (WeightedSum(value=21996, variance=21996) with flow)


In [24]:
# print(type(h))

In [25]:
h1= compute_results[0]["GluGluHToBB"]["ExampleHistogram1"]

KeyError: 'GluGluHToBB'

In [ ]:
h2= compute_results[0]["GluGluHToBB"]["ExampleHistogram2"]

In [ ]:
h3= compute_results[0]["GluGluHToBB"]["ExampleHistogram3"]

In [ ]:
h4= compute_results[0]["GluGluHToBB"]["ExampleHistogram4"]

In [ ]:
h5= compute_results[0]["GluGluHToBB"]["ExampleHistogram5"]

In [ ]:
h[{'eta':sum, 'pt':sum, 'msoftdrop':sum}].plot1d(label="beta=0")
h1[{'msoftdrop1':sum}].plot1d(label="beta=1")
h2[{'msoftdrop2':sum}].plot1d(label="beta=2")
h3[{'msoftdrop3':sum}].plot1d(label="beta=-0.5")
plt.suptitle('ggF 2018')
plt.ylabel('Events')

plt.legend()


In [ ]:
h[{'eta':sum, 'pt':sum, 'n2':sum}].plot1d(label="beta=0")
h1[{'n2b':sum}].plot1d(label="beta=1")
h2[{'n2b2':sum}].plot1d(label="beta=2")
h3[{'n2b3':sum}].plot1d(label="beta=-0.5")
plt.suptitle('ggF 2018')
plt.ylabel('Events')

plt.legend()

In [ ]:
h[{'eta':sum, 'pt':sum, 'msoftdrop':sum}].plot1d(label="zcut=0.10")
h4[{'msoftdropz1':sum}].plot1d(label="zcut=0.05")
h5[{'msoftdropz2':sum}].plot1d(label="zcut=0.20")
plt.suptitle('ggF 2018')
plt.ylabel('Events')

plt.legend()

In [ ]:
h[{'eta':sum, 'pt':sum, 'n2':sum}].plot1d(label="zcut=0.10")
h4[{'n2z1':sum}].plot1d(label="zcut=0.05")
h5[{'n2z2':sum}].plot1d(label="zcut=0.20")
plt.ylabel('Events')


plt.legend()

In [ ]:
client.close

In [ ]:
client.close()

In [ ]:
# from coffea.analysis_tools import Weights

In [ ]:
# Weights?

In [ ]:
# def analysis(df):
#     processor_instance = msdProcessor(isMC=True)
#     result = processor_instance.process(df)
#     return result

In [ ]:
# df = compute

In [ ]:
# results = dask.delayed(analysis)(df)
# final_result = dask.compute(results)

In [ ]:
#result_compute

In [ ]:
tg_result = dask.compute(tg)
rep_result = dask.compute(rep)